In [1]:
import numpy as np
import scipy.stats as ss
import elfi

In [2]:
def fun(x, batch_size, random_state):
    return np.sum(x, keepdims=True, axis=1)

In [3]:
t1 = elfi.Prior(ss.multivariate_normal, [0, 0])
sim = elfi.Simulator(fun, t1, observed=np.array([[0]]))
d = elfi.Distance('euclidean', sim)

In [4]:
res = elfi.Rejection(sim, batch_size=10)

In [5]:
res.sample(10)

Method: Rejection
Number of samples: 10
Number of simulations: 1000
Threshold: -3.2
Invalid input parameters. Probably you use multidimensional parameters in defining distributions
 unsupported format string passed to numpy.ndarray.__format__